# 🛡️ AI Fraud Risk Analyzer — Colab Edition

Welcome to your interactive fraud detection playground!  
This notebook blends rule-based scoring with LLM-powered explanations to help you analyze financial transactions in real time.

---

## 🚀 What’s Inside

- 🔍 **Fraud Scoring Engine** — Evaluates risk based on amount, merchant, method, and timing
- 🧠 **LLM Explanation** — Uses a large language model to explain why a transaction might be risky
- 🎲 **Simulation Module** — Generates synthetic transactions for testing
- 📊 **Insights Dashboard** — View fraud rates and transaction summaries

---

## 🧠 Powered By

- 🤖 Hugging Face Transformers  
- 📚 Mistral-7B-Instruct (or your preferred model)  
- 🐍 Python + Pandas  
- 🧪 Google Colab

---

> ⚠️ Note: If you're using a gated model like `mistralai/Mistral-7B-Instruct-v0.2`, make sure you've requested access and authenticated with your Hugging Face token.

Let’s dive in and make fraud detection explainable, teachable, and deployable — all from your browser.

In [1]:
!pip install transformers --quiet

from datetime import datetime
import random
import pandas as pd
from transformers import pipeline

explainer = pipeline("text-generation", model="tiiuae/falcon-7b-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
transactions = [
    {"id": 1, "user_id": "alice", "amount": 150.0, "merchant": "amazon", "method": "credit_card"},
    {"id": 2, "user_id": "bob", "amount": 2500.0, "merchant": "suspicious_store", "method": "wire_transfer"},
    {"id": 3, "user_id": "charlie", "amount": 50.0, "merchant": "starbucks", "method": "debit_card"},
]

In [3]:
def score_transaction(tx):
    amount = tx["amount"]
    merchant = tx["merchant"].lower()
    method = tx["method"].lower()
    hour = datetime.now().hour

    score = 0
    reasons = []

    if amount > 10000: score += 0.5; reasons.append("💰 Very high amount")
    elif amount > 5000: score += 0.3; reasons.append("💳 High amount")
    elif amount > 1000: score += 0.1; reasons.append("💵 Medium amount")

    if any(w in merchant for w in ["suspicious", "fraud", "scam", "fake"]):
        score += 0.4; reasons.append("🏪 Suspicious merchant")

    if method in ["wire_transfer", "crypto", "gift_card"]:
        score += 0.2; reasons.append("💳 Risky method")

    if hour < 6 or hour > 23:
        score += 0.1; reasons.append("🌙 Odd hour")

    score += random.uniform(0, 0.15)
    score = min(score, 1.0)
    is_fraud = score > 0.7

    return {
        "score": round(score, 3),
        "is_fraud": is_fraud,
        "reasons": reasons or ["✅ No significant risk factors"]
    }

In [4]:
def explain_transaction(tx):
    prompt = (
        f"Analyze this transaction:\n"
        f"User: {tx['user_id']}\n"
        f"Amount: ${tx['amount']}\n"
        f"Merchant: {tx['merchant']}\n"
        f"Method: {tx['method']}\n"
        f"Is this risky? Explain.\n"
    )
    return explainer(prompt)[0]["generated_text"].split("Explain.")[-1].strip()

In [5]:
def simulate(n=5):
    users = ["alice", "bob", "charlie", "diana", "eve"]
    merchants = ["amazon", "walmart", "starbucks", "suspicious_store", "fake_shop"]
    methods = ["credit_card", "debit_card", "wire_transfer", "crypto"]

    results = []
    for _ in range(n):
        tx = {
            "user_id": random.choice(users),
            "amount": round(random.uniform(10, 8000), 2),
            "merchant": random.choice(merchants),
            "method": random.choice(methods)
        }
        result = score_transaction(tx)
        tx.update(result)
        tx["explanation"] = explain_transaction(tx)
        results.append(tx)

    return pd.DataFrame(results)

In [6]:
df = simulate(2)
display(df)

fraud_rate = df["is_fraud"].mean() * 100
print(f"🚨 Fraud Rate: {fraud_rate:.1f}%")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


,user_id,amount,merchant,method,score,is_fraud,reasons,explanation
0,diana,3206.17,walmart,debit_card,0.199,False,[💵 Medium amount],"Yes, this transaction is considered risky as i..."
1,bob,4866.45,suspicious_store,crypto,0.806,True,"[💵 Medium amount, 🏪 Suspicious merchant, 💳 Ris...","Yes, this transaction is considered risky. Cry..."


🚨 Fraud Rate: 50.0%
